In [3]:
!curl -o frase.txt -L https://raw.githubusercontent.com/ccouto/testecolab/main/frase.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17  100    17    0     0     47      0 --:--:-- --:--:-- --:--:--    47


In [2]:
fIn = open('frase.txt',"r")
print(fIn.readlines())

['isto é uma frase']


In [ ]:
print('ok')

![img](https://raw.githubusercontent.com/ccouto/testecolab/main/notation.png)

In [1]:
from teste import callme

callme('ola')


o valor de x é ola
